In [1]:
import json
import pandas as pd

from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

In [2]:
dir_root = "Data/"
name = "COVID Super Expert"
df_meta = pd.read_csv(f"{dir_root}/metadata.csv")
df_meta = df_meta[~df_meta["abstract"].isnull()]
df_meta = df_meta[~df_meta["arxiv_id"].isnull()]
df_meta = df_meta[~df_meta["pmc_json_files"].isnull()]
df_meta.info()

/tmp/ipykernel_385691/1433955684.py:3: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta = pd.read_csv(f"{dir_root}/metadata.csv")


<class 'pandas.core.frame.DataFrame'>
Index: 526 entries, 681854 to 1055479
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cord_uid          526 non-null    object 
 1   sha               525 non-null    object 
 2   source_x          526 non-null    object 
 3   title             526 non-null    object 
 4   doi               526 non-null    object 
 5   pmcid             526 non-null    object 
 6   pubmed_id         510 non-null    object 
 7   license           526 non-null    object 
 8   abstract          526 non-null    object 
 9   publish_time      526 non-null    object 
 10  authors           511 non-null    object 
 11  journal           525 non-null    object 
 12  mag_id            0 non-null      float64
 13  who_covidence_id  0 non-null      object 
 14  arxiv_id          526 non-null    object 
 15  pdf_json_files    525 non-null    object 
 16  pmc_json_files    526 non-null    object

In [3]:
df_meta = df_meta[["cord_uid", "title", "abstract", "arxiv_id", "pmc_json_files"]]
df_meta.info()
df_meta.head()

<class 'pandas.core.frame.DataFrame'>
Index: 526 entries, 681854 to 1055479
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cord_uid        526 non-null    object
 1   title           526 non-null    object
 2   abstract        526 non-null    object
 3   arxiv_id        526 non-null    object
 4   pmc_json_files  526 non-null    object
dtypes: object(5)
memory usage: 24.7+ KB


,cord_uid,title,abstract,arxiv_id,pmc_json_files
681854,xqze9mo8,Tracing contacts to evaluate the transmission ...,"We investigate, through a data-driven contact ...",2106.05404,document_parses/pmc_json/PMC8712527.xml.json
682125,ra24i31z,Facial expressions can detect Parkinson’s dise...,A prevalent symptom of Parkinson’s disease (PD...,2012.05373,document_parses/pmc_json/PMC8417264.xml.json
682895,jpxnj5j8,Replacing quarantine of COVID-19 contacts with...,The quarantine of identified close contacts ha...,2108.12938,document_parses/pmc_json/PMC8901648.xml.json
683021,c3sn7is3,A stochastic model for the influence of social...,The short-term economic consequences of the cr...,2103.15577,document_parses/pmc_json/PMC8502768.xml.json
683935,7akri5ky,A model for the Twitter sentiment curve,Twitter is among the most used online platform...,2011.05933,document_parses/pmc_json/PMC8049311.xml.json


In [4]:
full_text = []
for i in range(len(df_meta)):
    file = df_meta["pmc_json_files"].values[i]
    with open(f"{dir_root}{file}","r") as f:
        publ = json.load(f)
    
    text = [df_meta["abstract"].values[i]]
    text += [item["text"] for item in publ["body_text"]]
    # text = "\n".join(text)
    
    if "\\usepackage" in text: 
        full_text.append(None)
    else:
        full_text.append(text)

df_meta["full_text"] = full_text
df_meta = df_meta[~df_meta["full_text"].isnull()]
df_meta.to_json(f"{dir_root}/covid_subset.json",orient="records")
df_meta.info()
df_meta.head()

<class 'pandas.core.frame.DataFrame'>
Index: 526 entries, 681854 to 1055479
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cord_uid        526 non-null    object
 1   title           526 non-null    object
 2   abstract        526 non-null    object
 3   arxiv_id        526 non-null    object
 4   pmc_json_files  526 non-null    object
 5   full_text       526 non-null    object
dtypes: object(6)
memory usage: 44.9+ KB


,cord_uid,title,abstract,arxiv_id,pmc_json_files,full_text
681854,xqze9mo8,Tracing contacts to evaluate the transmission ...,"We investigate, through a data-driven contact ...",2106.05404,document_parses/pmc_json/PMC8712527.xml.json,"[We investigate, through a data-driven contact..."
682125,ra24i31z,Facial expressions can detect Parkinson’s dise...,A prevalent symptom of Parkinson’s disease (PD...,2012.05373,document_parses/pmc_json/PMC8417264.xml.json,[A prevalent symptom of Parkinson’s disease (P...
682895,jpxnj5j8,Replacing quarantine of COVID-19 contacts with...,The quarantine of identified close contacts ha...,2108.12938,document_parses/pmc_json/PMC8901648.xml.json,[The quarantine of identified close contacts h...
683021,c3sn7is3,A stochastic model for the influence of social...,The short-term economic consequences of the cr...,2103.15577,document_parses/pmc_json/PMC8502768.xml.json,[The short-term economic consequences of the c...
683935,7akri5ky,A model for the Twitter sentiment curve,Twitter is among the most used online platform...,2011.05933,document_parses/pmc_json/PMC8049311.xml.json,[Twitter is among the most used online platfor...


In [5]:
tokenizer_opt = AutoTokenizer.from_pretrained("facebook/opt-6.7b")
tokenizer_gptj = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer_llama = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
tokenizer_llama2 = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

title = []
cord_uid = []
abstract = []
arxiv_id = []
paragraphs = []
pmc_json_files = []

for text in tqdm(df_meta["full_text"].values):
    for p in text:        
        tl = len(tokenizer_llama2(p, return_tensors="pt")["input_ids"][0])
        tg = len(tokenizer_gptj(p, return_tensors="pt")["input_ids"][0])
        if tl <= 256 and tg <= 256:
            paragraphs.append(p)
            title.append(df_meta["title"].values[i])
            cord_uid.append(df_meta["cord_uid"].values[i])
            abstract.append(df_meta["abstract"].values[i])
            arxiv_id.append(df_meta["arxiv_id"].values[i]) 
            pmc_json_files.append(df_meta["pmc_json_files"].values[i])
        else:
            sentences = sent_tokenize(p)
            sub_p = ""
            for s in sentences:
                sub_p_tmp = sub_p
                sub_p_tmp += s
                sub_plenl = len(tokenizer_llama2(sub_p_tmp, return_tensors="pt")["input_ids"][0])
                sub_pleng = len(tokenizer_gptj(sub_p_tmp, return_tensors="pt")["input_ids"][0])
                if sub_plenl > 256 or sub_pleng > 256:
                    paragraphs.append(sub_p)
                    title.append(df_meta["title"].values[i])
                    cord_uid.append(df_meta["cord_uid"].values[i])
                    abstract.append(df_meta["abstract"].values[i])
                    arxiv_id.append(df_meta["arxiv_id"].values[i]) 
                    pmc_json_files.append(df_meta["pmc_json_files"].values[i])
                    sub_p = ""
                else:
                    sub_p = sub_p_tmp

df_paragraphs = pd.DataFrame({"paragraph" : paragraphs})
df_paragraphs["title"] = title
df_paragraphs["cord_uid"] = cord_uid
df_paragraphs["abstract"] = abstract
df_paragraphs["arxiv_id"] = arxiv_id
df_paragraphs["pmc_json_files"] = pmc_json_files
df_paragraphs["paragraph"] = df_paragraphs["paragraph"].apply(lambda x: x.strip())
df_paragraphs["words_count"] = df_paragraphs["paragraph"].apply(lambda x: len(x.split(" ")))
df_paragraphs["t_opt_count"] = df_paragraphs["paragraph"].apply(lambda x: len(tokenizer_opt(x, return_tensors="pt")["input_ids"][0]))
df_paragraphs["t_gptj_count"] = df_paragraphs["paragraph"].apply(lambda x: len(tokenizer_gptj(x, return_tensors="pt")["input_ids"][0]))
df_paragraphs["t_llama_count"] = df_paragraphs["paragraph"].apply(lambda x: len(tokenizer_llama(x, return_tensors="pt")["input_ids"][0]))
df_paragraphs["t_llama2_count"] = df_paragraphs["paragraph"].apply(lambda x: len(tokenizer_llama2(x, return_tensors="pt")["input_ids"][0]))
df_paragraphs

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0% 0/526 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (4877 > 2048). Running this sequence through the model will result in indexing errors
100% 526/526 [01:26<00:00,  6.05it/s]


,paragraph,title,cord_uid,abstract,arxiv_id,pmc_json_files,words_count,t_opt_count,t_gptj_count,t_llama_count,t_llama2_count
0,"We investigate, through a data-driven contact ...",Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,148,190,189,209,209
1,Human mobility is crucial to understanding the...,Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,105,172,171,212,212
2,Public transportation is one of the main forms...,Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,129,166,165,188,188
3,"Here, we define two data-driven mathematical m...",Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,122,172,171,209,209
4,We propose a contact tracing model using two d...,Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,82,124,123,133,133
...,...,...,...,...,...,...,...,...,...,...,...
27805,The histories of the different variables are s...,Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,45,68,67,73,73
27806,The paper recognizes the importance of social ...,Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,77,236,235,244,244
27807,The extended model also studies the effects of...,Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,84,244,243,250,250
27808,We have also extended the SIR model to include...,Modelling the influence of progressive social ...,cjqhn392,The basic Susceptible-Infected-Recovered (SIR)...,2008.11962,document_parses/pmc_json/PMC7509834.xml.json,59,225,224,221,221


In [6]:
df_paragraphs.describe()

,arxiv_id,words_count,t_opt_count,t_gptj_count,t_llama_count,t_llama2_count
count,2.781000e+04,27810.000000,27810.000000,27810.000000,27810.000000,27810.000000
mean,2.008120e+03,86.508774,130.932147,129.932147,145.188565,145.188637
std,4.547555e-13,51.663903,71.695722,71.695722,79.247675,79.247748
min,2.008120e+03,1.000000,1.000000,0.000000,1.000000,1.000000
25%,2.008120e+03,45.000000,75.000000,74.000000,83.000000,83.000000
50%,2.008120e+03,88.000000,140.000000,139.000000,156.000000,156.000000
75%,2.008120e+03,130.000000,194.000000,193.000000,218.000000,218.000000
max,2.008120e+03,208.000000,257.000000,256.000000,256.000000,256.000000


In [7]:
df_paragraphs.to_json(f"{dir_root}/covid_subset_paragraphs.json",orient="records")